In [18]:
from mcp_agent.app import MCPApp
from mcp_agent.agents.agent import Agent
from mcp_agent.workflows.llm.augmented_llm import RequestParams
from mcp_agent.workflows.llm.augmented_llm_openai import OpenAIAugmentedLLM
from mcp_agent.config import (
    Settings,
    MCPSettings,
    LoggerSettings,
    OpenAISettings
)

import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", default=None)
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", default=None)


if OPENAI_API_KEY is None or OPENAI_BASE_URL is None:
    raise ValueError("請設定環境變數 OPENAI_API_KEY 和 OPENAI_BASE_URL")

print("OPENAI_API_KEY:", OPENAI_API_KEY[:5] + "...")
print("OPENAI_BASE_URL:", OPENAI_BASE_URL)

OPENAI_API_KEY: sk-28...
OPENAI_BASE_URL: http://localhost:4000/v1


In [19]:
settings = Settings(
        execution_engine="asyncio",
        logger=LoggerSettings(transports=["file"], level="debug"),
        mcp=MCPSettings(
            servers={"TDX_BIKE":{
                "name": "Bike Rental Service",
                "transport": "sse",
                "url": "http://localhost:8002/sse",
            }}
        ),
        openai=OpenAISettings(
            api_key=OPENAI_API_KEY,
            base_url=OPENAI_BASE_URL,
        )
    )

In [ ]:
instruction="""
你是一名公共自行車的服務人員，你可以回答有關公共自行車的問題。

當使用者問你「臺中科技大學附近有沒有自行車站？」時，你可以使用 TData 格式的過濾條件查詢鄰近站點，條件如下：  
contains(StationName/Zh_tw, '臺中科技大學')

當使用者問你「xxx站點目前是否還有自行車可供租借？」或「該站點剩餘幾台車？」時，請依下列流程操作：
1. 先使用 TData 查詢該站的基本資訊（如站名模糊搜尋）取得 StationID
2. 再用該 StationID 作為查詢條件，透過 TDX_BIKE_BikeApi_Availability 查詢即時可借車輛數  
   範例如：$filter = "StationID eq '500605061'"

請務必依照工具需求提供格式正確的參數，並避免使用模糊、不完整或結構不正確的資訊。若使用者輸入不足，請進行追問後再執行查詢。

若 MCP 工具回傳資料為空，請清楚告知使用者。

After receiving a response from the tool, always prepend your final reply with the agent and tool name in square brackets.  
For example: [transportation_agent] [get_bike_station_status]
"""

In [ ]:
# instruction = """
# 你是一名公共自行車的服務人員，你可以回答有關公共自行車的問題。

# 當使用者問你「臺中科技大學附近有沒有自行車站？」時，你可以使用TData格式過濾器來查詢臺中科技大學附近的自行車站，過濾條件\"contains(StationName/Zh_tw, '臺中科技大學')\

# 當使用者問你「xxx站點目前是否還有自行車可供租借的資訊」時，你可以使用TDX_BIKE_BikeApi_Availability的$filter="StationID eq '{查詢到的站點ID}'"來查詢該車站的剩餘車位數量。
# """

In [ ]:
app = MCPApp(
    name="My Agent App",
    settings=settings
)
async with app.run() as mcp_agent_app:
    bike_agent = Agent(
        name="Bike Agent",
        instruction=instruction,
        server_names=["TDX_BIKE"]
    )

    async with bike_agent:
        result_tools = await bike_agent.list_tools()
        print("tools數量", len(result_tools.tools)-1)
        llm = await bike_agent.attach_llm(OpenAIAugmentedLLM)

In [ ]:
### 情境一
result = await llm.generate_str(
    message="臺中科技大學附近是否有公共自行車站點，如果有請提供該站點目前是否還有自行車可供租借的資訊。",
    request_params=RequestParams(model="gpt-4o", use_history=False)
)
print(result)

In [ ]:
### 情境二
result = await llm.generate_str(
    message="太平國小附近是否有公共自行車站點，如果有請提供該站點目前是否還有自行車可供租借的資訊。",
    request_params=RequestParams(model="gpt-4o", use_history=False)
)
print(result)